In [ ]:
# | default_exp classes.DomoSandbox

In [2]:
# | exporti
from dataclasses import dataclass, field

from typing import Optional
import datetime as dt
import asyncio
import httpx
import uuid
from fastcore.basics import patch_to

# import importlib
# import json
# import uuid
# import time

import domolibrary.utils.DictDot as util_dd
import domolibrary.client.DomoAuth as dmda
import domolibrary.client.DomoError as de
import domolibrary.routes.sandbox as sandbox_routes

import domolibrary.classes.DomoLineage as dmdl

In [10]:
# | export

from dataclasses import dataclass, field
import aiohttp
import asyncio
import datetime as dt

from pprint import pprint


class InvalidRepositoryError(Exception):
    pass


@dataclass
class DomoRepository:
    id: str
    name: str
    last_updated_dt: dt.datetime
    commit_dt: dt.datetime
    commit_version: str
    auth: dmda.DomoAuth = None
    content_page_id_ls: list[str] = None
    content_card_id_ls: list[str] = None
    content_dataflow_id_ls: list[str] = None
    content_view_id_ls: list[str] = None

    def __post_init__(self):
        self.lineage = dmdl.DomoLineage(
                                        parent=self)

    @classmethod
    def _from_json(cls, obj, auth=None):

        import dateutil.parser as dtut

        dd = util_dd.DictDot(obj)

        return cls(
            id=dd.id,
            auth=auth,
            name=dd.name,
            content_page_id_ls=dd.repositoryContent.pageIds,
            content_card_id_ls=dd.repositoryContent.cardIds,
            content_dataflow_id_ls=dd.repositoryContent.dataflowIds,
            content_view_id_ls=dd.repositoryContent.viewIds,
            last_updated_dt=dtut.parse(dd.updated).replace(tzinfo=None),
            commit_dt=dtut.parse(dd.lastCommit.completed).replace(tzinfo=None),
            commit_version=dd.lastCommit.commitName
        )

    @classmethod
    async def get_from_id(cls, repository_id: str, auth: dmda.DomoFullAuth):
        res = await sandbox_routes.get_repo_from_id(repository_id=repository_id, auth=auth)

        if res.status == 404:
            raise InvalidRepositoryError

        return cls._from_json(res.response, auth=auth)

    def convert_lineage_to_dataframe(self, return_raw: bool = False):
        import pandas as pd
        import re

        flat_lineage_ls = self.lineage._flatten_lineage()

        output_ls = [{'sandbox_id': self.id,
                      'sandbox_name': self.name,
                      'version': self.commit_version,
                      'commit_dt': self.commit_dt,
                      'last_updated_dt': self.last_updated_dt,
                      'entity_type': row.get('entity_type'),
                      'entity_id': row.get('entity_id')
                      } for row in flat_lineage_ls]

        if return_raw:
            return output_ls

        return pd.DataFrame(output_ls)


@dataclass
class DomoSandbox:

    @classmethod
    async def get_repositories(cls, auth):
        res = await sandbox_routes.get_shared_repos(auth)

        if res.status != 200:
            return None

        domo_repos = [DomoRepository._from_json(
            obj, auth=auth) for obj in res.response.get('repositories')]

        return domo_repos

Sample of call to get Sandbox repositories

In [11]:
import os
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community", domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

sandbox_repos = await DomoSandbox.get_repositories(
   auth=token_auth
)

from pprint import pprint

pprint(sandbox_repos)

[DomoRepository(id='abd4cb2b-ec72-4373-a471-2a04db500713',
                name='Landing Page_PROD',
                last_updated_dt=datetime.datetime(2023, 6, 15, 21, 9, 0, 244000),
                commit_dt=datetime.datetime(2023, 6, 15, 21, 17, 17, 188000),
                commit_version='v2',
                auth=DomoTokenAuth(domo_instance='domo-community',
                                   token_name='token_auth',
                                   is_valid_token=True,
                                   url_manual_login='https://domo-community.domo.com/auth/index?domoManualLogin=true'),
                content_page_id_ls=[],
                content_card_id_ls=[],
                content_dataflow_id_ls=[226],
                content_view_id_ls=[]),
 DomoRepository(id='bc9b553c-ea02-4fce-ba63-2a4a63b455ef',
                name='Landing Page_QA',
                last_updated_dt=datetime.datetime(2023, 6, 15, 21, 11, 39, 941000),
                commit_dt=datetime.datetime(2023, 6,